In [55]:




def lire_fichier(chemin):
  with open(chemin, "r", encoding="utf-8") as f: 
    chaine = f.read()
  return chaine

from nltk.tokenize import word_tokenize

def decouper_en_mots(chaine):
  #liste_mots = chaine.split()
  liste_mots = word_tokenize(chaine)
  return liste_mots


In [56]:
import glob, re

dic_langues = {}
for chemin in glob.glob("corpus_multi/*/appr/*"):
  dossiers = chemin.split("/")#Sur Linux/Mac: chemin.split("/")
  langue = dossiers[1]
  if langue not in dic_langues:
    dic_langues[langue] = {}
  chaine = lire_fichier(chemin)
  mots = chaine.split()
  for m in mots:
    if m not in dic_langues[langue]:
      dic_langues[langue][m] = 1
    else:
      dic_langues[langue][m] += 1


In [57]:
print(len(dic_langues))
L = dic_langues.keys()
print(L)
print(list(L))
#1/0
dic_models = {}
for langue, dic_effectifs in dic_langues.items():
  paires = [[effectif, mot] for mot, effectif in  dic_effectifs.items()]
  liste_tri = sorted(paires)[-10:]#les 10 mots fréquents
  dic_models[langue] = [mot for effectif, mot in liste_tri]

import json
w = open("models_nltk.json", "w")
w.write(json.dumps(dic_models, indent = 2, ensure_ascii=False))
w.close()

22
dict_keys(['en', 'et', 'it', 'mt', 'pt', 'sk', 'sl', 'bg', 'es', 'da', 'nl', 'lt', 'hu', 'pl', 'fr', 'fi', 'lv', 'el', 'ro', 'sv', 'de', 'cs'])
['en', 'et', 'it', 'mt', 'pt', 'sk', 'sl', 'bg', 'es', 'da', 'nl', 'lt', 'hu', 'pl', 'fr', 'fi', 'lv', 'el', 'ro', 'sv', 'de', 'cs']


In [58]:
with open("models_nltk.json") as f:
    dic_modeles = json.load(f)
exactitude = 0
Nb_fichiers = len(glob.glob("corpus_multi/*/test/*"))
stats = {}
stats_pred = {}
dic_results = {}
pas_decision = 0
for chemin in glob.glob("corpus_multi/*/test/*"):
    dossiers = chemin.split("/")
    langue_reelle = dossiers[1]
    if langue_reelle not in dic_results:
        dic_results[langue_reelle] = {"VP" : 0, "FP":0, "FN" :0}
    stats.setdefault(langue_reelle, 0)
    stats[langue_reelle]+=1
    chaine = lire_fichier(chemin)
    liste_mots = decouper_en_mots(chaine)
    dic = {}
    for m in liste_mots:
        if m not in dic:
            dic[m] = 1
        else:
            dic[m]+=1
    paires = []
    for mot, effectif in dic.items():
        paires.append([effectif, mot])
    mots_freq = sorted(paires)[-10:]
    mots_OK = []
    for effectif, mot in mots_freq:
        mots_OK.append(mot)
    dic_en_commun = {}
    for langue, modele in dic_modeles.items():
        mots_en_commun = set(mots_OK).intersection(set(modele))
        dic_en_commun[langue] = len(mots_en_commun)
    diagnostics = []
    for langue, NB_communs in dic_en_commun.items():
        diagnostics.append([NB_communs, langue])
    langue_predite = sorted(diagnostics)[-1][1]
    if sorted(diagnostics)[-1][0]==sorted(diagnostics)[-2][0]:
        pas_decision+=1
        continue
    if langue_predite not in dic_results:
        dic_results[langue_predite] = {"VP" : 0, "FP":0, "FN" :0}
    if langue_reelle == langue_predite:
        exactitude+=1
        dic_results[langue_reelle]["VP"]+=1
    else:
        dic_results[langue_reelle]["FN"]+=1
        dic_results[langue_predite]["FP"]+=1
    stats_pred.setdefault(langue_predite, 0)
    stats_pred[langue_predite]+=1
#print(dic_results)
#print(stats)
#print(stats_pred)

print("Exactitude absolue : ", exactitude)
print("Pas de décision : ", pas_decision)
print("Exactitude relative : ", exactitude/Nb_fichiers)
print("Exactitude relative filtrée : ", exactitude/(Nb_fichiers-pas_decision))



Exactitude absolue :  1134
Pas de décision :  74
Exactitude relative :  0.9204545454545454
Exactitude relative filtrée :  0.9792746113989638


In [48]:
for langue, dic in dic_results.items():
    rappel = dic["VP"]/(dic["VP"]+dic["FN"])
    precision=  dic["VP"]/(dic["VP"]+dic["FP"])
    F = (2*rappel*precision)/(precision+rappel)
    moyenne = (rappel+ precision)/2
    print(langue, "\t",round(rappel, 3),"\t", round(precision, 3),"\t",round(moyenne,3), "\t", round(F,3))

en 	 1.0 	 0.718 	 0.859 	 0.836
et 	 0.875 	 1.0 	 0.938 	 0.933
fi 	 0.982 	 0.902 	 0.942 	 0.94
it 	 0.982 	 1.0 	 0.991 	 0.991
mt 	 0.982 	 1.0 	 0.991 	 0.991
pt 	 0.982 	 1.0 	 0.991 	 0.991
sk 	 0.982 	 0.873 	 0.928 	 0.924
sl 	 0.982 	 0.982 	 0.982 	 0.982
bg 	 0.982 	 1.0 	 0.991 	 0.991
es 	 0.982 	 1.0 	 0.991 	 0.991
da 	 0.982 	 1.0 	 0.991 	 0.991
nl 	 0.982 	 1.0 	 0.991 	 0.991
lt 	 0.964 	 1.0 	 0.982 	 0.982
ro 	 0.982 	 0.965 	 0.974 	 0.973
hu 	 0.982 	 1.0 	 0.991 	 0.991
pl 	 0.964 	 1.0 	 0.982 	 0.982
fr 	 0.982 	 1.0 	 0.991 	 0.991
lv 	 0.982 	 1.0 	 0.991 	 0.991
el 	 0.982 	 1.0 	 0.991 	 0.991
sv 	 0.982 	 1.0 	 0.991 	 0.991
de 	 0.982 	 1.0 	 0.991 	 0.991
cs 	 0.804 	 1.0 	 0.902 	 0.891


In [8]:
#Correction d'une autre manière

liste_langues = dic_models.keys() 
dic_resultats ={lg: {"VP":0, "FP":0, "FN":0} for lg in liste_langues}
erreurs= {}
for chemin in glob.glob("corpus_multi/*/test/*"):
  dossiers = chemin.split("/")
  langue = dossiers[1]
  chaine = lire_fichier(chemin)
  mots = chaine.split()
  dic_freq_texte = {}
  for m in mots:
    if m not in dic_freq_texte:
      dic_freq_texte[m] = 1
    else:
      dic_freq_texte[m] += 1
  paires = [[effectif, mot] for mot, effectif in  dic_freq_texte.items()]
  liste_tri = sorted(paires)[-10:]#les 10 mots fréquents
  plus_frequents = set([mot for effectif, mot in liste_tri])
  liste_predictions = []
  for langue_ref, model in dic_models.items():
    mots_communs = set(model).intersection(plus_frequents)
    NB_mots_communs = len(mots_communs)
    liste_predictions.append([NB_mots_communs, langue_ref])
  liste_predictions = sorted(liste_predictions)
  lg_pred = liste_predictions[-1][1]
  if lg_pred == langue:
    dic_resultats[langue]["VP"]+=1
  else:
    dic_resultats[langue]["FN"]+=1
    dic_resultats[lg_pred]["FP"]+=1
    erreurs.setdefault((langue, lg_pred), 0)
    erreurs[(langue, lg_pred)] +=1

In [9]:

for langue, infos in dic_resultats.items():
  VP = infos["VP"]
  FP = infos["FP"]
  FN = infos["FN"]
  if VP!=0:
    rappel = VP/(VP+FN)
    precision=VP/(VP+FP)
    f_mesure = (2*rappel*precision)/(precision+rappel)
  else:
    rappel, precision, f_mesure = 0, 0, 0
  print("%s : rappel =%f, précision=%f et f-mesure=%f"%(langue,rappel, precision, f_mesure))

for paire, NB_erreurs in erreurs.items():
  print("Pour la paire %s : %i erreurs"%(str(paire), NB_erreurs))

en : rappel =1.000000, précision=0.717949 et f-mesure=0.835821
et : rappel =0.875000, précision=1.000000 et f-mesure=0.933333
it : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
mt : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
pt : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
sk : rappel =0.982143, précision=0.873016 et f-mesure=0.924370
sl : rappel =0.982143, précision=0.982143 et f-mesure=0.982143
bg : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
es : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
da : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
nl : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
lt : rappel =0.964286, précision=1.000000 et f-mesure=0.981818
hu : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
pl : rappel =0.964286, précision=1.000000 et f-mesure=0.981818
fr : rappel =0.982143, précision=1.000000 et f-mesure=0.990991
fi : rappel =0.982143, précision=0.901639 et f-mesure=0